# Missing value imputation

In [1]:
import pandas as pd
import numpy as np

In [2]:
Country_code=["AUS", "BRA", "CAN", "CHN", "GBR", "IND", "JPN", "SGP", "USA"]
Manufacture = pd.ExcelFile('cleaned/Manufacturing.xlsx')
Service = pd.ExcelFile("cleaned/Service.xlsx")
Manufacture_df=[]
Service_df=[]
for x in Country_code:
    Manufacture_df.append(pd.read_excel(Manufacture, x).replace('\.+', np.nan, regex=True))
    Service_df.append(pd.read_excel(Service, x).replace('\.+', np.nan, regex=True))

In [3]:
def Mean_imputation(df):
    df = df.copy()[df.Year>1979]
    df.fillna(df.mean(),inplace=True)
    return df
def Median_imputation(df):
    df = df.copy()[df.Year>1979]
    df.fillna(df.median(),inplace=True)
    return df
def LOCF(df):
    df = df.copy()[df.Year>1979]
    df=df.ffill()
    return df
def NOCB(df):
    df = df.copy()[df.Year>1979]
    df=df.bfill()
    return df
def Rolling_Statistics(df):
    df = df.copy()[df.Year>1979]
    df=df.ewm(halflife=4).mean()
    return df
def interpolation(df):
    df = df.copy()[df.Year>1979].astype(float)
    df=df.interpolate('linear',axis=0)
    return df
Missing_technique=["Mean", "Median", "LOCF", "NOCB", "Rolling statistics", "Interpolate"]

In [4]:
def create_df_output(df, Country_code, Missing_technique, k):
    if k ==1:
        output_path = 'cleaned/Manufacturing_filled.xlsx'
    elif k==0:
        output_path = "cleaned/Service_filled.xlsx"
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for x in range(len(Country_code)):
            for y in range(len(Missing_technique)):
                if y == 0:
                    df_temp = Mean_imputation(df[x])
                elif y == 1:
                    df_temp = Median_imputation(df[x])
                elif y == 2:
                    df_temp = LOCF(df[x])
                elif y == 3:
                    df_temp = NOCB(df[x])
                elif y == 4:
                    df_temp = Rolling_Statistics(df[x])
                elif y == 5:
                    df_temp = interpolation(df[x])
                df_temp.dropna(axis = 1) \
                       .to_excel(writer, sheet_name=Country_code[x] + ' ' + Missing_technique[y], index=False)

In [5]:
create_df_output(Manufacture_df, Country_code, Missing_technique, 1)
create_df_output(Service_df, Country_code, Missing_technique, 0)